In [14]:
import dask as dd
import pandas as pd

In [ ]:
final_cluster_pd = pd.read_parquet(r"C:\Users\muthuvu\Videos\10X-MLE-master\notebook\output_clusters_single\per_user_top4_clsuters_noduplicates_summary_peruser.parquet")
final_cluster_pd.drop("summary_per_user", axis=1, inplace=True)
final_cluster_pd.head()

muid   event_name device_os  channel  \
0  f2ffe5a9-165d-4021-8707-d197d7ce8e20       scroll   android  browser   
1  f2ffe5a9-165d-4021-8707-d197d7ce8e20     pageview   android  browser   
2  f2ffe5a9-165d-4021-8707-d197d7ce8e20       scroll   android  browser   
3  f2ffe5a9-165d-4021-8707-d197d7ce8e20  viewcontent   android  browser   
4  28e2c09c-0eb2-45dd-9256-89e568856848     pageview   android      app   

  category location  cluster  
0    books       CA        0  
1    books       CA       19  
2  fashion       CA       60  
3    books       CA       16  
4    books       PK       17

In [16]:
cluster_summary = []
cluster_nums = []
cluster_size = []
for i in range(146):
    cluster_i = final_cluster_pd[final_cluster_pd["cluster"] == i]
    mode_vals = cluster_i[['event_name', 'device_os', 'channel', 'category']].mode().values
    try:
        cluster_summary.append(mode_vals[0])  # take the first row (flattened)
        cluster_nums.append(i)
        cluster_size.append(len(cluster_i))
    except:
        continue

summary_df = pd.DataFrame({
    "cluster": cluster_nums,
    "cluster_mode1": cluster_summary,
    "cluster_size": cluster_size,
})

summary_df["summary"] = summary_df["cluster_mode1"].apply(
    lambda x: f"users {x[0]} {x[3]} using {x[1]} on {x[2]}"
)
summary_df

cluster                        cluster_mode1  cluster_size  \
0          0    [scroll, android, browser, books]         39264   
1          1       [viewcontent, ios, app, books]         55242   
2          2     [pageview, android, app, coffee]         27492   
3          3   [add_to_cart, android, app, books]         31332   
4          4      [pageview, ios, browser, books]         42007   
..       ...                                  ...           ...   
137      139      [purchase, desktop, app, snack]            22   
138      140  [add_to_cart, desktop, app, coffee]          1489   
139      141       [search, android, app, beauty]             2   
140      142      [search, android, app, grocery]             2   
141      143   [scroll, desktop, browser, coffee]          2129   

                                           summary  
0      users scroll books using android on browser  
1         users viewcontent books using ios on app  
2       users pageview coffee using android on app  
3     users add_to_cart books using android on app  
4        users pageview books using ios on browser  
..                                             ...  
137      users purchase snack using desktop on app  
138  users add_to_cart coffee using desktop on app  
139       users search beauty using android on app  
140      users search grocery using android on app  
141   users scroll coffee using desktop on browser  

[142 rows x 4 columns]

In [ ]:
final_cluster_pd = final_cluster_pd.merge(summary_df, on="cluster")
summary_df["muids"] = summary_df["cluster"].apply(lambda x: final_cluster_pd[final_cluster_pd["cluster"]==x]["muid"].values)
summary_df.to_csv(r"C:\Users\muthuvu\Videos\10X-MLE-master\notebook\output_clusters_single\final_cluster_summary_df.csv", index=False)

cluster                        cluster_mode1  cluster_size  \
0          0    [scroll, android, browser, books]         39264   
1          1       [viewcontent, ios, app, books]         55242   
2          2     [pageview, android, app, coffee]         27492   
3          3   [add_to_cart, android, app, books]         31332   
4          4      [pageview, ios, browser, books]         42007   
..       ...                                  ...           ...   
137      139      [purchase, desktop, app, snack]            22   
138      140  [add_to_cart, desktop, app, coffee]          1489   
139      141       [search, android, app, beauty]             2   
140      142      [search, android, app, grocery]             2   
141      143   [scroll, desktop, browser, coffee]          2129   

                                           summary  \
0      users scroll books using android on browser   
1         users viewcontent books using ios on app   
2       users pageview coffee using android on app   
3     users add_to_cart books using android on app   
4        users pageview books using ios on browser   
..                                             ...   
137      users purchase snack using desktop on app   
138  users add_to_cart coffee using desktop on app   
139       users search beauty using android on app   
140      users search grocery using android on app   
141   users scroll coffee using desktop on browser   

                                                 muids  
0    [f2ffe5a9-165d-4021-8707-d197d7ce8e20, ce20411...  
1    [6a2da504-2309-4350-8f5c-25f09f2d668d, 6a2da50...  
2    [28e2c09c-0eb2-45dd-9256-89e568856848, 89f1918...  
3    [89f1918d-92dc-4ef9-a848-1d5cd7514024, 1180c7c...  
4    [bb8fc4a2-0d6d-4f16-a0a6-0412d3326dd4, 797039d...  
..                                                 ...  
137  [ac7aab48-c831-4e6a-bfea-f8f460a7e0ab, 23158dc...  
138  [302784c1-2602-43c7-a269-de0ce40a3c1b, 302784c...  
139  [1f7328e9-17e1-49e9-a764-0dba7f7915df, d92b249...  
140  [ccd91654-d7c5-4b9f-87fe-8f65e895ebd4, e35fc84...  
141  [1401e335-56c7-4ef2-bc9a-4798e9b83f97, b567400...  

[142 rows x 5 columns]

In [20]:
(
    final_cluster_pd.groupby("cluster", group_keys=False)
    .apply(lambda x: x.sample(min(len(x), 10), random_state=42))
    .reset_index(drop=True)
).to_csv(r"C:\Users\muthuvu\Videos\10X-MLE-master\notebook\output_clusters_single\final_cluster_pd.csv", index=False)


C:\Users\muthuvu\AppData\Local\Temp\ipykernel_3524\2989762663.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(min(len(x), 10), random_state=42))
